In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import xgboost as xgb

In [2]:
dfAll = pd.read_csv('../../Data/All_non_missing_v2.csv')
raw_predictors = dfAll.columns.tolist()[1:-13]
raw_predictors[-1]

'750X1384'

In [14]:
group_list

[['210X3',
  '210X18',
  '210X90',
  '210X91',
  '210X182',
  '261X388',
  '330X709',
  '330X808',
  '330X818'],
 ['210X9', '312X85'],
 ['210X13',
  '210X14',
  '210X15',
  '210X16',
  '210X33',
  '210X35',
  '311X94',
  '311X182',
  '261X554',
  '261X740'],
 ['210X19',
  '210X214',
  '220X3',
  '220X471',
  '220X476',
  '220X478',
  '220X480',
  '220X481',
  '220X482',
  '220X483',
  '220X486',
  '220X487',
  '220X488',
  '220X489',
  '310X3',
  '310X38',
  '311X7',
  '311X8',
  '311X33',
  '311X43',
  '311X69',
  '311X118',
  '311X157',
  '311X206',
  '261X343',
  '261X529',
  '261X715',
  '312X253',
  '312X255',
  '312X256',
  '312X257',
  '312X259',
  '312X261',
  '312X262',
  '312X263',
  '312X265',
  '312X267',
  '312X268',
  '312X269',
  '312X335',
  '312X356',
  '312X357',
  '312X358',
  '312X359',
  '312X373',
  '312X374',
  '312X375',
  '312X376',
  '312X379',
  '312X380',
  '312X381',
  '312X382',
  '312X383',
  '312X388',
  '330X906',
  '330X1053',
  '340X51',
  '340X59',
 

In [15]:
base_list = raw_predictors.copy()
group_list = []
for i in range(len(raw_predictors)-1):
    if raw_predictors[i] in base_list:
        tmp_list = [raw_predictors[i]]
    else:
        continue
    for j in range(i+1,len(raw_predictors)):
        if ((dfAll[raw_predictors[i]]-dfAll[raw_predictors[j]])==0).sum()>=20 and raw_predictors[j] in base_list:
            tmp_list.append(raw_predictors[j])
            base_list.remove(raw_predictors[j])
    if len(tmp_list)>=2:
        group_list.append(tmp_list)

In [4]:
range_df = pd.DataFrame({'min':dfAll[raw_predictors].min(),'max':dfAll[raw_predictors].max()},index=raw_predictors)

In [6]:
range_df.sort_values(['max','min'],inplace=True)

In [10]:
range_df['min'].nunique()

1851

In [12]:
range_df['range'] = 1
range_df.groupby(['max','min'])['range'].sum()

max            min          
-8.000000e+01  -1.201000e+02     3
-1.270000e+01  -1.680000e+01     1
-1.250000e+01  -1.440000e+01     1
-1.200000e+01  -2.200000e+01     1
-2.000000e+00  -2.410000e+02     1
-9.000000e-01  -3.600000e+00     1
-1.000000e-01  -7.970000e+01     1
               -2.000000e-01     1
 0.000000e+00  -9.920000e+01     1
               -1.200000e+01     1
               -7.000000e+00     1
               -6.000000e+00     1
               -5.500000e+00     1
               -4.000000e+00     5
               -3.000000e+00     5
               -2.000000e+00     1
               -1.000000e+00     3
               -2.840000e-01     1
               -1.000000e-01    15
 9.000000e-07   1.000000e-10     1
 1.900000e-06   1.000000e-10     1
 2.600000e-06   1.000000e-10     1
 3.000000e-06   1.000000e-10     1
 3.800000e-06   1.000000e-10     1
 4.200000e-06   1.000000e-10     2
 6.800000e-06   1.000000e-10     1
 8.100000e-06   1.000000e-10     1
 1.200000e-05   1.000000e-

In [13]:
(range_df.groupby(['max','min'])['range'].sum()>1).sum()

274

In [14]:
(range_df.groupby(['max','min'])['range'].sum()>2).sum()

108

In [19]:
tmp = range_df.groupby(['max','min'])['range'].sum()
tmp.reset_index(inplace=True)  
tmp

TypeError: Cannot reset_index inplace on a Series to create a DataFrame

In [25]:
tmp = tmp.reset_index()
tmp =tmp.loc[tmp['range']>1]
tmp

,max,min,range
0,-8.000000e+01,-1.201000e+02,3
13,0.000000e+00,-4.000000e+00,5
14,0.000000e+00,-3.000000e+00,5
16,0.000000e+00,-1.000000e+00,3
18,0.000000e+00,-1.000000e-01,15
24,4.200000e-06,1.000000e-10,2
29,3.400000e-05,1.000000e-10,2
50,9.800000e-04,1.000000e-10,2
51,1.000000e-03,1.000000e-10,3
52,1.100000e-03,1.000000e-10,7


In [32]:
range_list =[]
for i in tmp.index:
    tmp_list = range_df.loc[(range_df['min']==tmp.loc[i,'min'])&(range_df['max']==tmp.loc[i,'max'])].index.tolist()
    range_list.append(tmp_list)
range_list

[['330X629', '330X631', '330X634'],
 ['312X364', '312X365', '340X61', '344X144', '344X145'],
 ['312X337', '312X338', '312X340', '344X204', '344X205'],
 ['312X313', '312X341', '340X101'],
 ['400X133',
  '400X135',
  '400X151',
  '400X152',
  '400X154',
  '400X156',
  '400X157',
  '400X158',
  '400X159',
  '400X160',
  '420X135',
  '420X156',
  '420X157',
  '420X159',
  '420X160'],
 ['261X385', '261X572'],
 ['261X387', '261X573'],
 ['311X109', '311X196'],
 ['261X382', '261X568', '261X754'],
 ['311X195', '311X197', '261X381', '261X383', '261X567', '261X569', '261X753'],
 ['210X76', '210X127', '210X140'],
 ['210X57', '210X125'],
 ['210X65', '210X138'],
 ['210X29', '210X98', '210X99'],
 ['210X66', '210X139'],
 ['261X554', '261X740'],
 ['210X14', '210X34'],
 ['520X323', '520X324', '520X325', '520X326', '520X327'],
 ['400X132',
  '400X134',
  '400X136',
  '400X138',
  '400X139',
  '400X140',
  '400X141',
  '400X142',
  '420X115',
  '420X117',
  '420X120',
  '420X123',
  '420X124',
  '420X132'

In [34]:
for i in range(len(range_list)):
    dfAll['std_group_%d'%i] = dfAll[range_list[i]].std(axis=1)

In [36]:
grp = ['ID']
for var in dfAll:
    if 'std_group_' in var:
        grp.append(var)
dfAll[grp]

,ID,std_group_0,std_group_1,std_group_2,std_group_3,std_group_4,std_group_5,std_group_6,std_group_7,std_group_8,...,std_group_264,std_group_265,std_group_266,std_group_267,std_group_268,std_group_269,std_group_270,std_group_271,std_group_272,std_group_273
0,ID001,0.000000e+00,0.894427,1.095445,0.00000,0.050709,0.000000e+00,0.000000e+00,1.414143e-06,0.000000e+00,...,0.57735,0.000000,0.707107,0.577350,0.707107,0.000000,0.000000,0.707107,0.000000e+00,0.0
1,ID002,0.000000e+00,2.000000,1.643168,0.00000,0.048795,9.892424e-08,1.060589e-06,2.375879e-06,1.582193e-06,...,0.00000,0.000000,0.000000,0.577350,0.707107,0.000000,0.707107,0.000000,0.000000e+00,0.0
2,ID003,0.000000e+00,0.894427,0.836660,0.00000,0.048795,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.00000,0.000000,0.000000,0.577350,0.707107,0.000000,0.000000,0.000000,0.000000e+00,0.0
3,ID004,0.000000e+00,2.000000,1.643168,0.00000,0.050709,1.837771e-07,1.484924e-06,2.686935e-06,1.514318e-06,...,0.50000,0.000000,0.707107,0.577350,1.414214,0.000000,0.000000,0.000000,0.000000e+00,0.0
4,ID005,0.000000e+00,0.894427,0.547723,0.00000,0.050709,1.060589e-06,2.050610e-06,2.191960e-06,3.569734e-06,...,0.50000,0.000000,0.000000,0.577350,0.000000,0.000000,0.707107,2.121320,0.000000e+00,0.0
5,ID006,0.000000e+00,1.095445,0.547723,0.00000,0.050709,7.777467e-07,7.778175e-07,1.696986e-06,2.642442e-06,...,0.50000,0.000000,0.000000,0.577350,0.707107,0.000000,0.000000,1.414214,0.000000e+00,0.0
6,ID007,0.000000e+00,1.673320,1.303840,0.00000,0.051640,0.000000e+00,0.000000e+00,1.696986e-06,0.000000e+00,...,0.00000,0.000000,0.000000,0.577350,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0
7,ID010,0.000000e+00,1.095445,0.547723,0.00000,0.048795,1.972828e-08,1.201374e-07,7.778168e-05,0.000000e+00,...,0.50000,0.000000,0.707107,0.577350,2.121320,0.707107,0.000000,0.000000,0.000000e+00,0.0
8,ID011,0.000000e+00,2.190890,1.643168,0.00000,0.050709,8.484574e-07,1.979899e-06,1.979828e-06,5.859465e-07,...,0.50000,0.000000,0.000000,0.577350,2.828427,0.707107,0.000000,0.707107,0.000000e+00,0.0
9,ID012,0.000000e+00,1.673320,0.000000,0.00000,0.050709,0.000000e+00,3.040488e-06,3.323331e-06,1.327906e-06,...,0.50000,0.000000,0.000000,0.577350,1.414214,0.707107,0.000000,0.000000,0.000000e+00,0.0


In [37]:
dfAll[grp].to_csv('../../Cache/group_var_std.csv',index=False)

In [40]:
for i in range(len(range_list)):
    print(i)
    print(range_list[i])

0
['330X629', '330X631', '330X634']
1
['312X364', '312X365', '340X61', '344X144', '344X145']
2
['312X337', '312X338', '312X340', '344X204', '344X205']
3
['312X313', '312X341', '340X101']
4
['400X133', '400X135', '400X151', '400X152', '400X154', '400X156', '400X157', '400X158', '400X159', '400X160', '420X135', '420X156', '420X157', '420X159', '420X160']
5
['261X385', '261X572']
6
['261X387', '261X573']
7
['311X109', '311X196']
8
['261X382', '261X568', '261X754']
9
['311X195', '311X197', '261X381', '261X383', '261X567', '261X569', '261X753']
10
['210X76', '210X127', '210X140']
11
['210X57', '210X125']
12
['210X65', '210X138']
13
['210X29', '210X98', '210X99']
14
['210X66', '210X139']
15
['261X554', '261X740']
16
['210X14', '210X34']
17
['520X323', '520X324', '520X325', '520X326', '520X327']
18
['400X132', '400X134', '400X136', '400X138', '400X139', '400X140', '400X141', '400X142', '420X115', '420X117', '420X120', '420X123', '420X124', '420X132', '420X136', '420X150', '420X151', '420X152'